In [1]:
import numpy as np
from keras.src.legacy.preprocessing.image import ImageDataGenerator
%pip install opendatasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 3.3 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105137 sha256=06e3714df83709d7cf326c88817f0f00d976254115cc3cb2ff888a1ddc077c06
  Stored in directory: /home/sajjad/.cache/pip/wheels/d7/54/06/8a8f40cb39536605feb9acaacd0237a95eba39e5065e6392f4
Successfully built kaggle
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/usr/local/bin/slugify'
Consider using the `--user` option or check the permissions.

Note: you may need to restart the kernel to use updated packages.


In [3]:
import opendatasets as od
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf


2024-05-18 20:51:55.771069: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 20:51:56.742831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
od.download('https://www.kaggle.com/datasets/alxmamaev/flowers-recognition')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Your Kaggle Key: Dataset URL: https://www.kaggle.com/datasets/alxmamaev/flowers-recognition


100%|██████████| 225M/225M [02:30<00:00, 1.56MB/s] 


In [14]:
data_dir = 'flowers-recognition'


In [15]:
img_height, img_width = 224, 224
batch_size = 32

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                                   validation_split=0.2)

In [17]:
train_dataset = train_datagen.flow_from_directory(data_dir, target_size=(img_height, img_width), batch_size=batch_size,
                                                  class_mode='categorical', subset='training')

Found 3454 images belonging to 1 classes.


In [18]:
val_data = train_datagen.flow_from_directory(data_dir, target_size=(img_height, img_width), class_mode='categorical',
                                             batch_size=batch_size, subset='validation')

Found 863 images belonging to 1 classes.


In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
def classify_flowers(img):
    image = tf.image.resize(img, (img_height, img_width))
    image = image / 255.0
    prediction = model.predict(np.array([image]))
    class_names = ['daisy', 'dandelion', 'roses', 'sunflower', 'tulips']
    return {class_names[i]: float(prediction[0][i]) for i in range(len(class_names))}

In [28]:
import gradio as gr

iface = gr.Interface(
    fn=classify_flowers,
    inputs=gr.components.Image(),
    outputs=gr.components.Label(num_top_classes=5),
    title='Flower Classification',
    examples=['flowers-recognition/flowers/daisy/5547758_eea9edfd54_n.jpg']
)
iface.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
